In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True))

BASE_URL = "https://gigachat.devices.sberbank.ru/api/v1"
#base_gg = "https://router.huggingface.co/v1"
#"http://a6k2.dgx:34000/v1"

BASE_URL = "https://router.huggingface.co/v1"

#"qwen3-32b"
# gigachat/GigaChat
MODEL_NAME = os.getenv("MODEL_NAME", 'GigaChat')



MODEL_NAME_HF ='meta-llama/Llama-3.3-70B-Instruct'
MODEL_NAME_HF ='Qwen/Qwen3-32B'

MODEL_TEMPERATURE_HF =1

os.environ["OPENAI_API_KEY"] = os.getenv("GIGACHAT_AUTH")

In [2]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

In [13]:
from langchain_community.document_loaders import PyMuPDFLoader
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser, BaseOutputParser
from textwrap import dedent

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
import httpx
import json
import requests
import pandas as pd
import numpy as np
import re

from langchain_pymupdf4llm import PyMuPDF4LLMLoader, PyMuPDF4LLMParser
import pymupdf4llm

from trialmind.pubmed import pmid2papers, PubmedAPIWrapper, pmid2biocxml, parse_bioc_xml
from trialmind.api import StudyCharacteristicsExtraction, ScreeningCriteriaGeneration
from trialmind.retrievers import split_text_into_chunks

import requests
import uuid
from openai import OpenAI

import time
import extract
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### clinical trials

In [3]:
file_path = "docs/CC_215_L00_DL_fusion_fixed_oncobox_ru.pdf"
fin_condition, treatements_eng = extract.info_from_doc(file_path)
fin_condition, treatements_eng

('Colorectal cancer',
 ['Crizotinib',
  'Lorlatinib',
  'Necitumumab',
  'Nimotuzumab',
  'Panitumumab',
  'Cetuximab',
  'Brigatinib',
  'Ramucirumab',
  'Pomalidomide',
  'Toremifene',
  'Denosumab',
  'Duvelisib',
  'Inavolitib',
  'Bevacizumab',
  'Anastrozole',
  'Exemestane'])

1. загрузить док
2. взять 1-ую страницу; рег.выражениями взять диагноз
-- с ЛЛМ перевести на англ.
3. найти страницу с таблицей; или взять таблицы с PyMuPDFLoader
4. извлечь таблицу

In [63]:
' OR '.join(treatements_eng)

'Crizotinib OR Lorlatinib OR Necitumumab OR Nimotuzumab OR Panitumumab OR Cetuximab OR Brigatinib OR Ramucirumab OR Pomalidomide OR Toremifene OR Denosumab OR Duvelisib OR Inavolitib OR Bevacizumab OR Anastrozole OR Exemestane'

In [5]:
all_studies = extract.get_clinicaltrials(fin_condition, 
                                      #' OR '.join(treatements_eng), 
                                       treatements_eng[0],  
                                      max_studies=100)

In [6]:
all_studies

,results,id,title,status,conditions,study_type,phases,interventions,outcomes
0,False,NCT02465060,Targeted Therapy Directed by Genetic Testing i...,ACTIVE_NOT_RECRUITING,"[Advanced Lymphoma, Advanced Malignant Solid N...",INTERVENTIONAL,[PHASE2],"[adavosertib, afatinib, afatinib dimaleate, bi...",NaN
1,False,NCT03947385,Study of IDE196 in Patients With Solid Tumors ...,RECRUITING,"[Metastatic Uveal Melanoma, Cutaneous Melanoma...",INTERVENTIONAL,"[PHASE1, PHASE2]","[ide196, binimetinib, crizotinib]",NaN
2,False,NCT02568267,Basket Study of Entrectinib (RXDX-101) for the...,ACTIVE_NOT_RECRUITING,"[Breast Cancer, Cholangiocarcinoma, Colorectal...",INTERVENTIONAL,[PHASE2],[entrectinib],NaN
3,False,NCT06214793,Taletrectinib in Previously Treated Metastatic...,SUSPENDED,"[Breast Cancer, Metastatic Breast Cancer]",INTERVENTIONAL,[PHASE2],[taletrectinib],NaN
4,False,NCT03792568,ALK Inhibitor in Metastatic Colorectal Cancer ...,UNKNOWN,[Colorectal Cancer],INTERVENTIONAL,[NA],[],NaN
5,False,NCT05725200,Study to Investigate Outcome of Individualized...,RECRUITING,[Metastatic Colorectal Cancer],INTERVENTIONAL,[PHASE2],"[alectinib, cetuximab, crizotinib, dasatinib, ...",NaN
6,True,NCT02510001,MEK and MET Inhibition in Colorectal Cancer,COMPLETED,"[Solid Tumor, Colorectal Cancer]",INTERVENTIONAL,[PHASE1],"[pf-02341066, pd-0325901, binimetinib]","[{'type': 'PRIMARY', 'title': 'Maximal Tolerat..."


### outcome results
Взяли все результаты, отдельно сохранили испытания с результатами, и з них отбираем рез-ты только с unit==participants

filter by interventional study type?

In [251]:
outcome_participants = [obj for item_main in results_studies for obj in item_main['resultsSection']['outcomeMeasuresModule'
                                           ]['outcomeMeasures'
                                            ] if(obj.get('unitOfMeasure','') == 'Participants')] 
outcome_participants

[{'type': 'PRIMARY',
  'title': 'Clinical Response to Binimetinib Combined With PF-02341066',
  'description': 'To investigate response to treatment with RPII dose of Binimetinib with Crizotinib (PF-02341066), in patients with a) RASMT CRC or b) RASWT/cMET mut amplified CRC or c) RASWT/c-MET over-expressed CRC, as defined by stable, partially or completely responding disease, per Response Evaluation Criteria In Solid Tumors Criteria (RECIST v1.1) for target lesions and assessed by CT: Complete Response (CR), Disappearance of all target lesions; Partial Response (PR), \\>=30% decrease in the sum of the longest diameter of target lesions; Stable Disease (SD), Neither sufficient shrinkage to qualify for PR nor sufficient increase (\\>=20%) to qualify for Progressive Disease; Overall Response (OR) = CR + PR + SD',
  'populationDescription': 'Evaluable patients for the primary outcome are those patients who complete a response assessment after cycle 1 of treatment, or who progress early on 

In [273]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful medical assistant. Extract information about the efficiency statistics from each outcome measure. Give a concise description with percentages /no_think"),
    ("human", "{text}"),
    # ("ai", "Good.")
])

chain2 = prompt2 | llm_reasoning
response_res = await chain2.ainvoke({"text": outcome_participants[0]})
print(response_res.content)

<think>

</think>

Here is a concise summary of the efficiency statistics for the clinical response to Binimetinib combined with PF-02341066 (Crizotinib) in the Dose Expansion Phase:

- **Total Evaluable Patients**: 30 participants.
- **Stable Disease (SD)**: 7 patients (23.3%).
- **Progressive Disease (PD)**: 22 patients (73.3%).
- **Early Death from Malignant Disease**: 1 patient (3.3%).

These outcomes were assessed using RECIST v1.1 criteria after cycle 1 of treatment.


### results in a given format?

In [ ]:

class Info(BaseModel):
    diagnose: str = Field(..., description="База клинических испытаний")
    treatments: str = Field(..., description="Высшие позиции рейтинга таргетных препаратов")

parser = PydanticOutputParser(pydantic_object=Info)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract related information from the text provided by the user.
            Return the information in the following JSON-format.
            {fmt}
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

### papers

In [6]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]
pubmed_api_key= '6892c4129cef143ff92d11533848d2e0d908'#os.getenv("PUBMED_API_KEY", '')

In [5]:
search_api = PubmedAPIWrapper()
# page_size is the max number of records to return!!!! not pages!
tmp_inputs = {
        "page_size": 1,
        "keyword_map": {'conditions':[fin_condition], 
                        'treatments':[treatements_eng[0]]
                       },
        "keywords": {
            "OPERATOR": 'AND'
        }
}

time.sleep(.5)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, 
                                                      api_key=pubmed_api_key)
response

(['41268440'],
 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=(Colorectal cancer)+AND+(Crizotinib)&retmax=1&retmode=json&api_key=6892c4129cef143ff92d11533848d2e0d908',
 '67')

In [7]:
df_papers = pmid2papers(pmid_list=['41268440'],#response[0], 
                        api_key=pubmed_api_key)
df_papers[0].head()

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,41268440,Frontiers in pharmacology,2025,,,Case Report: HGF and NF1 mutations as putative...,"Case Reports, Journal Article","Zhitao Chen, Shan Luo, Yangjun Gu, Qiyong Li",Hepatocellular carcinoma (HCC) is a highly agg...


In [8]:
fin_condition, treatements_eng

('Colorectal cancer', ['Crizotinib', 'Lorlatinib'])

### extract results

In [9]:
fin_condition = 'Colorectal cancer'
treatements_eng =  ['Crizotinib','Lorlatinib',]

In [10]:
os.getenv("GIGACHAT_AUTH")

'MDE5YmM2MjctOWU5ZS03OWE0LWE0OGYtY2FhMTU4MzYxZDgzOjg2MjAwMDgzLThkNzMtNGU0ZS1hNzIxLTM3YWY5ZWY4ZmZlNg=='

### get sber api

In [139]:

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
# Создадим идентификатор UUID (36 знаков)
rq_uid = str(uuid.uuid4())
payload={
  'scope': 'GIGACHAT_API_PERS'
}
headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Accept': 'application/json',
  'RqUID': rq_uid,
  'Authorization': f'Basic {os.getenv("GIGACHAT_AUTH")}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)
os.environ["OPENAI_API_KEY"] = response.json()['access_token']

os.getenv("OPENAI_API_KEY")

D:\Programs\Anaconda\envs\llms\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'eyJjdHkiOiJqd3QiLCJlbmMiOiJBMjU2Q0JDLUhTNTEyIiwiYWxnIjoiUlNBLU9BRVAtMjU2In0.Z_GQStOEb6STz0PuIvi2c4hZzSijwZMB8XO6ONX3M3Kp2KAbSZJrOU76w4XZxfoD_VXFQVMwBKUKFii1fj2WPPbang57m8icdbqkMRUBEL71dyAHqIg1il6dXoAbKu4iI1Ps9guhPNrwU26pQ64eCJA4agcH2SL3qeMruYPz7lVOt7YrQS4O9cCUO7YnKDLa-BCffEEv_B_H-oGryLeJ_LnvPChX49dLXGyBRdFF9DQ6qI3eyZ9KsU4aSRYpeYiozapNPg0Isdi9_dQoX0jvluGS_SurQ-gmyiY7I3_DSgYHrIDqeGa8C0AsiROO7eHuYxOzOXUkPowp8pRQAMa7Yw.MWSJ9br9Ka7Lay1gHVCA2A.wH5v2iXQ3Qgk0Jzkz_S2UrjKE6YxvE37CDYnzqsYFI_G5Oh8qgun3_W1YJxACkqsx787825R30XwqMrDVYON12t8hYQJiw9uPYI8NmS3IOOuK5rKeILa95NdKxAfZmB5BJHfnDNJPLiTvNPATu7YldXQdHCBOd7sx1kyreqCuNJAuidGKEI9qB7vIBesN9nmA83ESQ2QiNMm-QKnqCu0LVm1iJAZ9BWDOEZuXkHvcSpeglp_M6fpWm5CkrdKZkPpExZUtX7Yg8qZ-ErNIem4wfSaNemAhAR9YlieUgwKyiKWPMMblc80iMq8MhDCH5OiBTzxtVJyVSmrRLkc-Iw5_jC7qSMuhiyLXHdFFO-Dn6xENp5pgB7wng5JpqE0eZw2wyBZ5LX1IkbGk_j6XG-REB-G6nyvYDzYYYPL8F7aYalov3BANykGScL57AG7VfXKgKKt5giG1LdBay3ImQKombw-byieqw1ttRw-jHRhlyAFncbmLYlS7UCaBvFLeW_XWzsG_Hnmv4ZBCo712njTbG_6yk61n-gTRIJTgZLCL0PHF

In [14]:
os.environ["BASE_URL"] = "https://gigachat.devices.sberbank.ru/api/v1"
os.environ["MODEL_NAME"] ='GigaChat'

In [15]:
os.getenv("BASE_URL")

'https://gigachat.devices.sberbank.ru/api/v1'

In [17]:
# test the apis

api = ScreeningCriteriaGeneration()
ecs = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    num_title_criteria=3,
    num_abstract_criteria=3,
    thinking=False,
)
ecs

call_openai https://gigachat.devices.sberbank.ru/api/v1
api_call_single https://gigachat.devices.sberbank.ru/api/v1/
```json
{
  "ELIGIBILITY_ANALYSIS": [
    "Patients must have colorectal cancer — Studies focusing on other types of cancers would not provide relevant data.",
    "Studies involving patients receiving crizotinib as an intervention — Other treatments or interventions would not contribute to the analysis.",
    "Comparative studies where crizotinib is compared against another treatment or placebo — Studies without comparison groups cannot assess effectiveness relative to alternatives.",
    "Studies reporting outcomes related to patient survival, response rates, toxicity, or quality of life — Only studies measuring relevant outcomes can inform decision-making.",
    "Published studies conducted after 2010 — Older studies may lack current evidence or methodology."
  ],
  "TITLE_CRITERIA": [
    "Does the title mention 'colorectal cancer'?",
    "Does the title include 'cri

{'title_criteria': ["Does the title mention 'colorectal cancer'?",
  "Does the title include 'crizotinib' or similar terms like 'Crizolta', 'PF-02341066'?",
  'Is there any indication in the title that the study compares crizotinib to another treatment?'],
 'content_criteria': ['Are the methods section and results clearly described in the paper?',
  'Do the results specifically report outcomes such as overall survival, progression-free survival, response rate, or adverse events?',
  'Was the study randomized controlled trial (RCT), prospective cohort study, or retrospective case-control study? — Preferably RCTs for higher reliability.'],
 'eligibility_analysis': ['Patients must have colorectal cancer — Studies focusing on other types of cancers would not provide relevant data.',
  'Studies involving patients receiving crizotinib as an intervention — Other treatments or interventions would not contribute to the analysis.',
  'Comparative studies where crizotinib is compared against anot

In [18]:
from trialmind.api import LiteratureScreening
api = LiteratureScreening()

papers = df_papers[0].iloc[:3]["Title"] + ": " + df_papers[0].iloc[:3]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()
papers

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC.\nWe present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance.\nNGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specific

In [20]:
'''
title_criteria = ["Does the title mention 'colorectal cancer'?",
"Does the title mention 'crizotinib'?",
"Is there any indication in the title that the study compares crizotinib to another treatment?"
]

content_criteria = ["Are the methods section and results clearly described in the paper?",
"Does the study specify the type of colorectal cancer (e.g., adenocarcinoma)?",
"Were the outcomes measured over a sufficient duration to assess long-term effects?"
  ]
'''
title_criteria = ecs['title_criteria']
content_criteria = ecs['content_criteria']
title_criteria+content_criteria

["Does the title mention 'colorectal cancer'?",
 "Does the title include 'crizotinib' or similar terms like 'Crizolta', 'PF-02341066'?",
 'Is there any indication in the title that the study compares crizotinib to another treatment?',
 'Are the methods section and results clearly described in the paper?',
 'Do the results specifically report outcomes such as overall survival, progression-free survival, response rate, or adverse events?',
 'Was the study randomized controlled trial (RCT), prospective cohort study, or retrospective case-control study? — Preferably RCTs for higher reliability.']

In [29]:
ec_pred = api.run(
    population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
    intervention = f"{treatements_eng[0]}",
    comparator = "",
    outcome = "",
    llm = os.getenv("MODEL_NAME"),
    criteria = title_criteria + content_criteria,
    papers = papers[:1], # make for the top-100 for demo
)
ec_pred

api_function_call_single https://gigachat.devices.sberbank.ru/api/v1/
[[{'role': 'user', 'content': '/no_think \n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly 6 responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extrapolating beyond the provided data, as accurate and reliable responses are crucial, and fabricating information (hallucinations) could lead to serious errors in the meta-analysis.\n\n# PICO FRAMEWORK #\n- P (Patient, Problem or Population): Patients with Colorectal cancer

[['UNCERTAIN',
  'UNCERTAIN',
  'UNCERTAIN',
  'UNCERTAIN',
  'UNCERTAIN',
  'UNCERTAIN']]

In [34]:
from openai import OpenAI

In [143]:
llm_reasoning = OpenAI(
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    http_client=httpx.Client(http2=True, verify=False)
)

In [100]:
from trialmind.prompts.screening import LITERATURE_SCREENING_FC
from trialmind.llm import _batch_inputs_to_messages

In [101]:
#"fmt": 
from typing import Union, Literal
class Criteria(BaseModel):
    decision: Union[str, Literal["YES", "NO", "UNCERTAIN"]]
    rationale: str = Field(..., description="a rationale for each decision")

class Criterias(BaseModel):
    criterias: list[Criteria]

parser = PydanticOutputParser(pydantic_object=Criterias)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Criteria": {"properties": {"decision": {"anyOf": [{"type": "string"}, {"enum": ["YES", "NO", "UNCERTAIN"], "type": "string"}], "title": "Decision"}, "rationale": {"description": "a rationale for each decision", "title": "Rationale", "type": "string"}}, "required": ["decision", "rationale"], "title": "Criteria", "type": "object"}}, "properties": {"criterias": {"items": {"$ref": "#/$defs/Criteria"}, "title": "Criterias", "type": "array"}}, "required": ["criterias"]}
```


In [117]:
population = f"Patients with {fin_condition} undergoing treatment with {treatements_eng[0]}",
intervention = f"{treatements_eng[0]}",
comparator = "",
outcome = "",
llm = os.getenv("MODEL_NAME"),
criteria = title_criteria + content_criteria,
papers = papers[:1], # make for the top-100 for demo
batch_inputs = []
batch_inputs.append({
            "P": population,
            "I": intervention,
            "C": comparator,
            "O": outcome,
            "paper_content": papers[0],
            "criteria_text": title_criteria + content_criteria,
            "num_criteria": 6,
            "fin_format":parser.get_format_instructions(),
    })
batch_inputs

[{'P': ('Patients with Colorectal cancer undergoing treatment with Crizotinib',),
  'I': ('Crizotinib',),
  'C': ('',),
  'O': ('',),
  'paper_content': (((((((['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC.\nWe present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify

In [134]:

prompt = ChatPromptTemplate.from_messages([('system',LITERATURE_SCREENING_FC),
                                          ('human', 'tell me')])
prompt

ChatPromptTemplate(input_variables=['C', 'I', 'O', 'P', 'criteria_text', 'fin_format', 'num_criteria', 'paper_content'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['C', 'I', 'O', 'P', 'criteria_text', 'fin_format', 'num_criteria', 'paper_content'], input_types={}, partial_variables={}, template='\n# CONTEXT #\nYou are a clinical specialist tasked with assessing research papers for inclusion in a meta-analysis based on specific eligibility criteria.\n\n# OBJECTIVE #\nEvaluate each criterion of a given paper to determine its eligibility for inclusion in the meta-analysis. Provide a list of decisions ("YES", "NO", or "UNCERTAIN") for each eligibility criterion. You must deliver exactly {num_criteria} responses.\n\n# IMPORTANT NOTE #\nIf the information within the provided paper content is insufficient to conclusively evaluate a criterion, you must opt for "UNCERTAIN" as your response. Avoid making assumptions or extra

In [145]:
llm_reasoning = ChatOpenAI(
    #api_base=os.getenv("BASE_URL"),
    base_url=os.getenv("BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model=os.getenv("MODEL_NAME"),
    streaming=True,
    http_async_client=httpx.AsyncClient(verify=False)
)


In [146]:
chain = (
    prompt 
    | llm_reasoning 
    | parser
)


response = await chain.ainvoke(batch_inputs[0])

In [148]:
print(response.model_dump_json(indent=4))

{
    "criterias": [
        {
            "decision": "NO",
            "rationale": "The title does not mention 'colorectal cancer'. It refers to hepatocellular carcinoma."
        },
        {
            "decision": "YES",
            "rationale": "The term 'crizotinib' is mentioned in the provided paper content."
        },
        {
            "decision": "NO",
            "rationale": "There is no comparison made between crizotinib and another treatment in the provided content."
        },
        {
            "decision": "YES",
            "rationale": "Methods and results sections are described in the paper, although details might be limited."
        },
        {
            "decision": "NO",
            "rationale": "The paper does not report specific outcomes such as overall survival, progression-free survival, response rate, or adverse events."
        },
        {
            "decision": "UNCERTAIN",
            "rationale": "It is unclear whether the study design is an

In [ ]:
response = await chain.ainvoke({"text": text, "fmt": parser.get_format_instructions()})
print(response)

In [83]:
ChatPromptTemplate.from_messages(LITERATURE_SCREENING_FC, batch_inputs)

ValueError: Unsupported template format: [{'P': ('Patients with Colorectal cancer undergoing treatment with Crizotinib',), 'I': ('Crizotinib',), 'C': ('',), 'O': ('',), 'paper_content': (((['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC.\nWe present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and 

In [61]:
from langchain_openai import ChatOpenAI

In [68]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system", 
        dedent(
            """
            You are a helpful assistant. 
            Your goal is to extract events related information from the text provided by the user.
            Return the information in the following JSON-format.
            ```json
            {{
                "events": [
                    {{
                        "date": "date of the event",
                        "time": "time of the event",
                        "reason": "why the activity was chosen"
                        "title": "title of the event",
                        "description": "description of the event"
                    }}
                ]
            }}
            ```
            if there is no events, return an empty list.
            /no_think
            """
        )
    ),
    (
        "human", 
        dedent(
            """
            Text: {text}"
            """
        )
    ),
])

chain = (
    prompt 
    | llm_reasoning 
    | StrOutputParser()
)


text = """
    We are planning to go to the cinema on 2025-11-04 at 14-00 to watch the movie 'The Dark Knight'. 
    Probably, we also will have dinner after the movie in the nearby restaurant two hours later.
"""
response = await chain.ainvoke({"text": text})
print(response)

```json
{
    "events": [
        {
            "date": "2025-11-04",
            "time": "14:00",
            "reason": "watching the movie 'The Dark Knight'",
            "title": "Cinema Visit",
            "description": "Going to the cinema to watch the movie 'The Dark Knight'."
        },
        {
            "date": "2025-11-04",
            "time": "16:00",
            "reason": "dinner after the movie",
            "title": "Dinner After Movie",
            "description": "Having dinner in a nearby restaurant after watching the movie."
        }
    ]
}
```


In [9]:
pmid_list = df_papers[0].PMID.values.tolist()
#['41213063',#'26451310',]

res = pmid2biocxml(pmid_list, api_key=pubmed_api_key)
#print(res)
res = [parse_bioc_xml(r) for r in res]
# transform the parsed xml into paper content
papers = []
for parsed in res:
    paper_content = []
    for parsed_ in parsed["passage"]:
        paper_content.append(parsed_['content'])
    paper_content = "\n".join(paper_content)
    papers.append(paper_content)


https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41268440/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/41213063/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40526090/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40211189/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908
https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/40369167/unicode?api_key=6892c4129cef143ff92d11533848d2e0d908


In [11]:
[len(i) for i in papers]

[1672, 1652, 27171, 2627, 46819]

In [15]:
splited = split_text_into_chunks(papers[1], chunk_size=600,
                                             chunk_overlap=300)
splited

['Case Report: HGF and NF1 mutations as putative bypass mechanisms of MET inhibitor resistance in hepatocellular carcinoma: a case study.',
 'Background: Hepatocellular carcinoma (HCC) is a highly aggressive liver cancer with poor prognosis, often associated with resistance to treatment. MET amplification has been identified as a potential therapeutic target, but resistance to MET inhibitors, such as crizotinib, remains a significant challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic',
 'challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression

In [12]:

api = StudyCharacteristicsExtraction()
extracted = api.run(
    papers=papers,
    fields=['Results, string, main results of the study regarding Crizotinib, usually about treatment effectiveness'],#api.DEFAULT_FIELDS,
    llm='Qwen/Qwen3-32B',#"openai-gpt-4o",
    chunk_size=600,
    chunk_overlap=300,
    thinking=False,
)
extracted

False


[[{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [2],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in MET-amplified HCC. Methods: We present a case of advanced HCC in a patient with MET amplification treated with crizotinib. After initial tumor regression, disease progression occurred. Genetic analysis using next-generation sequencing (NGS) was performed on biopsy samples taken before and after progression to identify mutations associated with resistance. Results: NGS revealed the loss of MET amplification and identified HGF and NF1 mutations as potential bypass mechanisms. Specifically,']}],
 [{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [5],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative rev

[[{'name': 'Crizotinib effectiveness',
   'value': 'More sensitive to low-risk group',
   'source_id': [4],
   'cited_blocks': ['performed in risk groups. Expression of signature genes in CRC cells was examined using quantitative reverse transcription polymerase chain reaction (qRT-PCR). There were 10 CRC-related RRAGs found. Prognostic models were built with RRAGs. Based on immune infiltration analysis, low-risk populations showed significantly greater levels of immune infiltration (P P ARHGEF17 mutation rate was 6%. Medications such as CGP-082996, Dasatinib, Erlotinib, and Salubrinal were more sensitive to high-risk group, whereas drugs such as FTI-277, DMOG, and Crizotinib were more sensitive to low-risk group.']}],
 [{'name': 'Crizotinib treatment effectiveness',
   'value': 'initial tumor regression followed by disease progression',
   'source_id': [1],
   'cited_blocks': ['challenge. This study aims to explore the molecular mechanisms underlying resistance to MET inhibitors in ME

In [38]:
llm_reasoning = ChatOpenAI(
    base_url=BASE_URL,
    api_key=HUGGINGFACE_HUB_TOKEN,
    model=MODEL_NAME_HF,
    streaming=True,
    temperature=0,
    #http_client=httpx_client
    http_async_client=httpx.AsyncClient(verify=False)
)

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("user", '/no_think {text}'),
    # ("ai", "Good.")
])

chain = prompt | llm_reasoning
response = await chain.ainvoke({"text": ff})

#fin_condition = response.content.strip('<think>\n\n</think>\n\n')
print(response.content)

<think>

</think>

```json
[
    {
        "name": "Crizotinib effectiveness",
        "value": "More sensitive to low-risk group",
        "source_id": [3]
    }
]
```
